In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

import config
from asxai.vectorDB.generate import OllamaManager

# import nest_asyncio
# nest_asyncio.apply()

import asyncio

In [2]:
from asxai.services.notebook.chat_api import app
from asxai.services.notebook.chat_workers import start_chat_worker

INFO:asxai.services.notebook.chat_api:Kafka topic 'notebook-queries' already exists.


Kafka broker up and ready


INFO:asxai.services.notebook.chat_workers:Query sent to Ollama for task_id=5c022ffb48a537cbf1e0/47f210eacb89414efd82


[{'role': 'user', 'content': 'how does the amygdala interacts with the hippocampus?'}]
[{'role': 'user', 'content': 'how does the amygdala interacts with the hippocampus?'}]


INFO:asxai.services.notebook.chat_workers:Streaming complete for task_id=5c022ffb48a537cbf1e0/47f210eacb89414efd82
INFO:asxai.services.notebook.chat_workers:Query sent to Ollama for task_id=5c022ffb48a537cbf1e0/47f210eacb89414efd82


[{'role': 'user', 'content': "how does the amygdala interacts with the hippocampus? If you can't find an accurate answer in the contex of the conversation, just return ASK QDRANT and nothing else."}]
[{'role': 'user', 'content': "how does the amygdala interacts with the hippocampus? If you can't find an accurate answer in the contex of the conversation, just return ASK QDRANT and nothing else."}]


INFO:asxai.services.notebook.chat_workers:Streaming complete for task_id=5c022ffb48a537cbf1e0/47f210eacb89414efd82


In [ ]:
import threading
import uvicorn

def start_chat_api():
    uvicorn.run(app, host="127.0.0.1", port=8100, reload=False)

api_thread = threading.Thread(target=start_chat_api, daemon=True)
api_thread.start()
print("Chat API started on http://127.0.0.1:8100")

Chat API started on http://127.0.0.1:8100


INFO:     Started server process [7447]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8100 (Press CTRL+C to quit)


INFO:     127.0.0.1:36550 - "GET /notebook/task_id?user_id=jul&notebook_id=my-notebook HTTP/1.1" 200 OK
gemma3:12b
INFO:     127.0.0.1:36556 - "POST /notebook/5c022ffb48a537cbf1e0/47f210eacb89414efd82/chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:36568 - "GET /notebook/5c022ffb48a537cbf1e0/47f210eacb89414efd82/stream HTTP/1.1" 200 OK
INFO:     127.0.0.1:56162 - "GET /notebook/task_id?user_id=jul&notebook_id=my-notebook HTTP/1.1" 200 OK
gemma3:12b
INFO:     127.0.0.1:56170 - "POST /notebook/5c022ffb48a537cbf1e0/47f210eacb89414efd82/chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:56186 - "GET /notebook/5c022ffb48a537cbf1e0/47f210eacb89414efd82/stream HTTP/1.1" 200 OK


In [ ]:
worker_thread = start_chat_worker()
print("Chat worker started.")

Chat worker started.


INFO:asxai.services.notebook.chat_workers:Kafka topic 'notebook-requests' already exists.


In [ ]:
import requests
import sseclient

# Input data
base_url = "http://127.0.0.1:8100"
user_id = "jul"
notebook_id = "my-notebook"
instruct_msg = " If you can't find an accurate answer to this question in the context of the conversation, \
     expand the question by rephrasing it in a more descriptive and complete sentence, \
     suitable for retrieving relevant scientific documents. Correct any spelling mistake when necessary. \
     Do not return anything else than just ASK_QDRANT: <query> where ,<query> is the expanded query"
message = "Using information from all 10 articles provided above — not just one — summarize how the amygdala interacts with the hippocampus."
message = "What is the main conclusion of article #9?"
message = "what type of recording technique do they use in article #7?"
message = "how does the amygdala interacts with the hippocampus?"
# message = "Reformulate the question into a context rich query to search through the QDRANT database. \
#            Return the results as a dictionnary with the reformulated query and the references of the articles we should search into in the following fields:\
#             \"qdrant_query\" \"article_refs\"\
#             If you can't find any relevant article reference in the conversation context, fill that field with 'null'."
# message = "How to identify the boundaries of the amygdala in MRI scans?"
# message = "what is nuclear fusion?"
model = 'gemma3:12b'#'deepseek-r1:32b' #

res = requests.get(f"{base_url}/notebook/task_id", params={
    "user_id": user_id,
    "notebook_id": notebook_id
})
task_id = res.json()["task_id"]
print("Task ID:", task_id)

Task ID: 5c022ffb48a537cbf1e0/47f210eacb89414efd82


In [9]:
res = requests.post(f"{base_url}/notebook/{task_id}/chat", json={
    "message": message + instruct_msg,
    "user_id": user_id,
    "notebook_id": notebook_id,
    "model": model
})
print("Chat submitted:", res.json())

Chat submitted: {'status': 'submitted', 'task_id': '5c022ffb48a537cbf1e0/47f210eacb89414efd82'}


In [10]:
# Stream response
stream_response = requests.get(f"{base_url}/notebook/{task_id}/stream", stream=True)
client = sseclient.SSEClient(stream_response)

print("\n📡 Streaming response:\n")
for event in client.events():
    print(event.data, end="\n", flush=True)
    if "<END_OF_MESSAGE>" in event.data:
        print(" ")
        break


📡 Streaming response:


🧑[You]:

how does the amygdala interacts with the hippocampus? If you can't find an accurate answer in the contex of the conversation, just return ASK QDRANT and nothing else.

🤖[asXai]

ASK QDRANT
<SEARCHING_QDRANT>


KeyboardInterrupt: 

In [102]:
s = "\n<END_OF_MESSAGE>\n"
s.replace('<END', '?BLA')
print(s)


<END_OF_MESSAGE>



In [20]:
import requests
res = requests.get(f"{base_url}/notebook/{task_id}/chat/final")
print(f"\n🧑[You]\n", res.json()["user"])
print("\n🤖[asXai]\n", res.json()["assistant"])


🧑[You]
 Based on the articles provided earlier, how can one identify the amygdala in MRI scans?

🤖[asXai]
 Based solely on the provided text, there's **no direct information on how to identify the amygdala in MRI scans.** The text focuses on measuring calcium dynamics within the amygdala using genetically modified mice and sophisticated analysis techniques. It doesn's describe how the amygdala would be visualized or identified using imaging modalities like MRI.

Here’s why, and what the text *does* say that's relevant:

*   **Genetically Modified Mice:** The experiments use genetically modified mice expressing calcium indicators. The research focuses on *activity* within the amygdala, not structural imaging.
*   **DeepEthogram & Pixel Data:** The analysis uses a machine-learning pipeline (DeepEthogram) to analyze pixel data of the mice’s behavior. This relates to tracking behavior, not visualizing brain structures.
*   **Focus on Activity:** The study's primary goal is to understand t

In [20]:
res = requests.get(f"{base_url}/notebook/{task_id}/history").json()
chat_history = res
for msg in chat_history:
    if msg['role'] == 'user':
        print(f"\n🧑[You]\n", msg["content"])
    elif msg['role'] == 'assistant':
        print("\n🤖[asXai]\n", msg["content"])


🧑[You]
 ---------
- ARTICLE REF: 5512607f33f02b67101c0bebe84b336cc8cae03f
  Title: AmygdalaGo-BOLT: an open and reliable AI tool to trace boundaries of human amygdala
  Authors Names: Quan Zhou,Bo Dong,Peng Gao,Jintao Wei,Jiale Xiao,Wei Wang,Peipeng Liang,Danhua Lin,Xiaoli Zuo,Hongjian He
Most relevant chunks: 'AmygdalaGo-BOLT3D: A boundary learning transformer for tracing human amygdala 1Center for Brain Imaging Science and Technology, Zhejiang University, Hangzhou, 310027, Zhejiang, China. 2College of Biomedical Engineering and Instrument Science, Zhejiang University, Hangzhou, 310027, China. 3State Key Laboratory of Cognitive Neuroscience and Learning, Beijing Normal University, Beijing, 100875, China. 4Faculty of Psychology, Beijing Normal University, Beijing, 100875, Beijing, China. 5School of Psychology, Capital Normal University, Beijing, 100190, China. 6School of Physics, Zhejiang University, Hangzhou, 310058, Zhejiang, China. 7State Key Laboratory of Brain-Machine Intelligenc

In [14]:
import requests

payload = {
    "user_id": "julien",
    "notebook_id": "neuroscience",
    "message": "Summarize the role of the prefrontal cortex"
}
task_id = "5c022ffb48a537cbf1e0/47f210eacb89414efd82"  # or use the /notebook/task_id endpoint to generate one

res = requests.post(f"http://localhost:8100/notebook/{task_id}/chat", json=payload)

In [10]:
url = f"http://localhost:8100/notebook/{task_id}/stream"
response = requests.get(url, stream=True)

In [12]:
url

'http://localhost:8100/notebook/5c022ffb48a537cbf1e0/47f210eacb89414efd82/stream'

In [4]:
import nest_asyncio
nest_asyncio.apply()

ollama_manager = OllamaManager()
messages = [{'role': 'user', 'content': 'Where is Paris', 'task_id': 0}]
async def ollama_chat(msg):
    print(msg[-1]['content'])
    streamer = asyncio.run(ollama_manager.client.chat(model=ollama_manager.model_name, messages=msg, stream=True))
    full_response = ""
    async for chunk in streamer:
        print(chunk["message"]["content"], end="")
        full_response += chunk["message"]["content"]
    return full_response

full_response = asyncio.run(ollama_chat(messages))
messages += [{'role': 'assistant', 'content': full_response, 'task_id': 0}]

messages += [{'role': 'user', 'content': "How many people live there?", 'task_id': 0}]
full_response = asyncio.run(ollama_chat(messages))

Where is Paris
Paris is the capital city of **France**.

Here's a more detailed breakdown of its location:

*   **Continent:** Europe
*   **Country:** France
*   **Region:** Île-de-France (Paris is the heart of this region)
*   **Coordinates:** 48.8566° N, 2.3522° E (approximately)

It's located in the north-central part of France, on the Seine River.



Do you want to know anything more specific about its location, like its proximity to other cities or landmarks?How many people live there?
Okay, let's break down the population of Paris. It gets a little complicated because "Paris" can refer to different areas:

**1. City of Paris (Ville de Paris):**

*   This is the core, central Paris, the area within the official city limits.
*   **Population:** Roughly **2.14 million** people (as of 2023 estimates).

**2. Greater Paris (Grand Paris):**

*   This is a larger metropolitan area including the city of Paris and surrounding suburbs. It's often used for planning and statistical purposes.


In [5]:
messages

[{'role': 'user', 'content': 'Where is Paris', 'task_id': 0},
 {'role': 'assistant',
  'content': "Paris is the capital city of **France**.\n\nHere's a more detailed breakdown of its location:\n\n*   **Continent:** Europe\n*   **Country:** France\n*   **Region:** Île-de-France (Paris is the heart of this region)\n*   **Coordinates:** 48.8566° N, 2.3522° E (approximately)\n\nIt's located in the north-central part of France, on the Seine River.\n\n\n\nDo you want to know anything more specific about its location, like its proximity to other cities or landmarks?",
  'task_id': 0},
 {'role': 'user', 'content': 'How many people live there?', 'task_id': 0}]

In [3]:
query = "what is the the role of the prefrontal cortex? I'm mostly interested in what Matteo Carandini and Keneth Harris said about that in their papers from 2016 to 2021?"
asyncio.run(ollama_manager.expand_parse_batch(queries=[query], options={'temperature': 0.5}))

{
  "cleaned_query": "What is the role of the prefrontal cortex in cognitive functions, and what specific insights regarding its function were presented?",
  "author_names": ["Carandini", "Harris"],
  "start_date": "2016-01-01",
  "end_date": "2021-12-31"
}
{'cleaned_query': 'What is the role of the prefrontal cortex in cognitive functions, and what specific insights regarding its function were presented?', 'author_names': ['Carandini', 'Harris'], 'start_date': '2016-01-01', 'end_date': '2021-12-31'}


{0: {'query': 'What is the role of the prefrontal cortex in cognitive functions, and what specific insights regarding its function were presented?',
  'authorName': ['Carandini', 'Harris'],
  'publicationDate_start': '2016-01-01',
  'publicationDate_end': '2021-12-31',
  'original_query': 'What is the role of the prefrontal cortex in cognitive functions, and what specific insights regarding its function were presented by Matteo Carandini and Kenneth Harris in their peer-reviewed publications between 2016 and 2021?'}}

In [5]:
import sys
sys.path.append("..")
sys.path.append("../src")

import config

import nest_asyncio
nest_asyncio.apply()

import asyncio
from ollama import AsyncClient, Client

In [17]:
syncclient = Client(
  host='http://localhost:11434'
)
models = syncclient.list()
models.models


[Model(model='gemma3:12b', modified_at=datetime.datetime(2025, 5, 19, 13, 40, 23, 526266, tzinfo=TzInfo(UTC)), digest='f4031aab637d1ffa37b42570452ae0e4fad0314754d17ded67322e4b95836f8a', size=8149190253, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='12.2B', quantization_level='Q4_K_M')),
 Model(model='mistral-small3.1:latest', modified_at=datetime.datetime(2025, 5, 19, 12, 41, 21, 742826, tzinfo=TzInfo(UTC)), digest='b9aaf0c2586a8ed8105feab808c0f034bd4d346203822f048e2366165a13f4ea', size=15486899116, details=ModelDetails(parent_model='', format='gguf', family='mistral3', families=['mistral3'], parameter_size='24.0B', quantization_level='Q4_K_M')),
 Model(model='deepseek-r1:7b', modified_at=datetime.datetime(2025, 5, 19, 12, 23, 28, 152446, tzinfo=TzInfo(UTC)), digest='0a8c266910232fd3291e71e5ba1e058cc5af9d411192cf88b6d30e92b6e73163', size=4683075271, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families

In [12]:
client = AsyncClient(
  host='http://localhost:11434'
)
async def chat(query):
  message = {'role': 'user', 'content': query}
  response = await AsyncClient().chat(model='llama3.1:8b', messages=[message])
  return response

syncclient = Client(
  host='http://localhost:11434'
)

In [23]:
syncclient.chat(model="gemma3:12b", messages=[{'role': 'user', 'content':'what is that?'}], options={'temperature': 0.5})

ChatResponse(model='gemma3:12b', created_at='2025-05-20T12:48:52.37206379Z', done=True, done_reason='stop', total_duration=2523802723, load_duration=78874714, prompt_eval_count=13, prompt_eval_duration=254114791, eval_count=110, eval_duration=2189200787, message=Message(role='assistant', content='Please provide me with more context! "That" is a pronoun that refers to something previously mentioned or something that can be seen or understood. \n\nTo tell you what "that" is, I need to know **what you\'re referring to.** \n\nFor example, tell me:\n\n* **What were you just talking about?**\n* **What are you looking at?**\n* **Where did you see "that"?**\n\n\n\nOnce you give me more information, I can tell you what "that" is.', images=None, tool_calls=None))

In [177]:
syncclient.pull('gemma3:12b')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [282]:
import re
import json

def extract_cleaned_query_and_metadata(text: str):
    try:
        start = text.index('{')
        end = text.rindex('}') + 1
        json_block = text[start:end]
        json_block = json_block.replace("None", "null")
        print(json_block)
        return json.loads(json_block)
    except (ValueError, json.JSONDecodeError):
        return {}

In [394]:
async def parse_query(query: str):
    parse_instruction = (
        "You are a helpful assistant that parse queries sent by users and only return the requested fields.\n"
        "Parse the following search query to extract the cleaned query and any metadata it contains."
        "Specifically, identify and separate:\n"
        "- author last names\n"
        "- Date to start searching from (formatted as %Y-%m-%d even if there's just a year mentioned)\n"
        "- Date to stop searching at (formatted as %Y-%m-%d even if there's just a year mentioned)\n"
        "Then reformulate the query without sections specifying authors names or dates.\n"
        "Return results as a dictionary with cleaned query and fields:\n"
        "{\"cleaned_query\" \"author_names\", \"start_date\", \"end_date\"}\n\n"
        "If you can't find answers for any of these fields just fill it with 'null'.\n"
        "Here is the original query:\n"
        f"'{query}'"
    )
    message = {'role': 'user', 'content': parse_instruction}
    response = await client.chat(model='gemma3:12b', messages=[message])
    return response['message']['content']

In [395]:
response = asyncio.run(parse_query("what is the role of the the prefrontal cortex? I'm mostly interested in what Carandini and Harris said about that in their papers from 2016 to 2021?"))

In [441]:
from dateutil.parser import parse
from datetime import datetime

def extract_parsed_fields(text: str):
    try:
        start = text.index('{')
        end = text.rindex('}') + 1
        json_block = text[start:end]
        json_block = json_block.replace("None", "null")
        dic = json.loads(json_block)
        default = datetime(1, 1, 1)  # defaults everything to Jan 1
        dt = parse("nov 2016", default=default)
        dt.strftime("%Y-%m-%d")
        norm_dic = {'query': [val for key, val in dic.items() if 'query' in key][0],
                    'authorName': [val for key, val in dic.items() if 'name' in key][0],
                    'publicationDate': [parse(val, default=default).strftime("%Y-%m-%d") for key, val in dic.items() if 'date' in key and val!='null']}
        norm_dic['publicationDate'] = sorted(norm_dic['publicationDate'])
        return norm_dic
    except (ValueError, json.JSONDecodeError):
        return {}


In [442]:
res = extract_parsed_fields(response)
res

{'query': 'What is the functional role of the prefrontal cortex in spatial navigation, and how does it contribute to navigational processes?',
 'authorName': 'null',
 'publicationDate': ['2016-11-01', '2021-12-31']}

In [398]:
async def expand_query(query: str):
    expand_instruction = (
        "You are a helpful assistant that expand queries sent by users to retrieve relevant scientific articles.\n"
        "Expand the following search query by rephrasing it in a more descriptive and complete sentence, "
        "suitable for retrieving relevant scientific documents."
        "Do not return anything else than just the expanded query.\n\n"
        "Here is the original query:\n"
        f"'{query}'"
    )
    message = {'role': 'user', 'content': expand_instruction}
    response = await client.chat(model='gemma3:12b', messages=[message])
    return response['message']['content']

In [435]:
response = asyncio.run(expand_query("what is the role of the the prefrontal cortex? Especially in the comntext of navigation.\n I'm only interested in papers from nov 2016 to 2021"))#("does the cerebellum influence place cell activity?"))
print(response)

What is the functional role of the prefrontal cortex in spatial navigation, and how does it contribute to navigational processes, as investigated in scientific publications published between November 2016 and 2021?


In [436]:
response = asyncio.run(parse_query(response))
res = extract_parsed_fields(response)
res

{'query': 'What is the functional role of the prefrontal cortex in spatial navigation, and how does it contribute to navigational processes?', 'authorName': 'null', 'publicationDate': []}


{'query': 'What is the functional role of the prefrontal cortex in spatial navigation, and how does it contribute to navigational processes?',
 'authorName': 'null',
 'publicationDate': []}

In [50]:
response = asyncio.run(chat('Why is the sky blue?'))
print(response['message']['content'])

The sky appears blue to us during the day because of a phenomenon called scattering. Scattering occurs when light from the sun interacts with the tiny molecules of gases in the Earth's atmosphere, such as nitrogen and oxygen.

Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When sunlight enters our atmosphere, it is made up of different colors of light, including red, orange, yellow, green, blue, indigo, and violet.
2. **Light interacts with gas molecules**: As the sunlight travels through the atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are randomly moving around due to thermal energy.
3. **Scattering occurs**: When light hits these gas molecules, it scatters in all directions. This scattering effect is more pronounced for shorter wavelengths of light, which correspond to blue and violet colors.
4. **Blue light is scattered more**: Since blue light has a shorter wavelength (around 450-495 nanometers), it is scat

In [40]:
client.show('llama3.1:8b')

ShowResponse(modified_at=datetime.datetime(2025, 5, 19, 9, 53, 42, 419687, tzinfo=TzInfo(UTC)), template='{{- if or .System .Tools }}<|start_header_id|>system<|end_header_id|>\n{{- if .System }}\n\n{{ .System }}\n{{- end }}\n{{- if .Tools }}\n\nCutting Knowledge Date: December 2023\n\nWhen you receive a tool call response, use the output to format an answer to the orginal user question.\n\nYou are a helpful assistant with tool calling capabilities.\n{{- end }}<|eot_id|>\n{{- end }}\n{{- range $i, $_ := .Messages }}\n{{- $last := eq (len (slice $.Messages $i)) 1 }}\n{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>\n{{- if and $.Tools $last }}\n\nGiven the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\n\nRespond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.\n\n{{ range $.Tools }}\n{{- . }}\n{{ end }}\nQuestion: {{ .Co

In [39]:
import torch

embeds = client.embed(model='llama3.1:8b', input=["what defines a species?"])
embeds

EmbedResponse(model='llama3.1:8b', created_at=None, done=None, done_reason=None, total_duration=13771611708, load_duration=13003152423, prompt_eval_count=5, prompt_eval_duration=None, eval_count=None, eval_duration=None, embeddings=[[-0.01936673, -0.016335309, 0.010500235, 0.012640309, -0.008382397, -0.0070123547, 0.010018112, -0.0062295045, 0.0076110763, -0.005243266, -0.0044069276, -0.009778513, -0.009192616, -0.0014424049, -4.105383e-07, 0.016774597, -0.022383217, -0.0002904055, -0.012512874, -0.0043807593, 0.019863166, -0.003627686, 0.0072256173, 0.01554155, -0.034741744, 0.0053445566, 0.013533217, 0.008982903, -0.011085375, -0.006031583, 0.003782622, 0.00787297, 0.022760691, 0.019526439, 0.053972542, -0.0017905999, 0.0031078288, 0.00977626, -0.0032927021, -0.013046428, 0.015456431, -0.0142626995, 0.010109175, 0.019162072, 0.015664848, -0.017895551, 0.0059482628, -0.0041574384, -0.002735969, -0.010922659, 0.011959368, 0.0074782264, 0.0009846555, 0.015870022, 0.011377611, -0.0107520

In [116]:
response = syncclient.chat(model='mistral-small3.1:latest', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue due to a phenomenon called Rayleigh scattering. As sunlight reaches Earth's atmosphere, it is made up of different colors, which are essentially different wavelengths of light. Shorter wavelengths (like blue and violet) are scattered in all directions more than other colors like red, yellow, or green, which have longer wavelengths.

Even though violet light is scattered even more than blue light, the sky appears blue, not violet, because our eyes are more sensitive to blue light and because the sun emits less violet light compared to blue light. Additionally, some of the violet light gets absorbed by the atmosphere, further tipping the balance in favor of blue.

So, the combination of these factors makes the sky appear blue to us on a clear day.


In [30]:
response = client.chat(model='deepseek-r1:7b', messages=[
  {
    'role': 'user',
    'content': 'reformulate the following query into a keyword rich one of more than 150 words. Just return the expanded query without any other sentence: what is the role of the amygdala?',
  },
])
print(response['message']['content'])

<think>
Okay, so I need to figure out how to answer this question about the amygdala's role. First, I remember that the amygdala is part of the brain, specifically in the almond-shaped area at the base of the skull. It's often called the "little brain." But what exactly does it do?

I think it's mainly involved with emotions like fear and anger. Maybe it processes things like sounds or sights that could be dangerous? I've heard terms like conditioned reflexes related to amygdala function. Also, during stress, the amygdala might get more active.

Wait, isn't there something about the prefrontal cortex interacting with the amygdala? Oh right, the amygdala helps regulate emotions and behaviors that prepare the body for action in case of danger. So when it detects a threat, it triggers fear responses. But how does it do that exactly?

I also recall that the amygdala is linked to memory, maybe storing past experiences with trauma or scary situations. That could explain why people are more f